<a href="https://colab.research.google.com/github/samanthajmichael/ml_project/blob/main/notebooks/Model_Prediction_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install tensorflow ffmpeg-python opencv-python matplotlib

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
import tensorflow as tf
import cv2
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Dense, Reshape, Input, Lambda, Lambda, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50

In [4]:
df = pd.read_pickle("/content/drive/MyDrive/ML Project/middle_15min_frames.pkl")

In [5]:
df.describe()

,frame_number
count,20700.000000
mean,92814.500000
std,5975.719622
min,82465.000000
25%,87639.750000
50%,92814.500000
75%,97989.250000
max,103164.000000


In [6]:
initial_learning_rate = 0.001
decay_steps = 1000
decay_rate = 0.9

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=decay_steps,
    decay_rate=decay_rate,
    staircase=True
)

In [7]:
def create_base_model(input_shape=(224, 224, 3)):
    base_model = ResNet50(weights=None, include_top=False, input_shape=input_shape)
    x = GlobalAveragePooling2D()(base_model.output)
    x = tf.keras.layers.BatchNormalization()(x)
    x = Dense(128, activation='relu')(x)
    # Replace Lambda layer with direct normalization in the forward pass
    outputs = tf.keras.layers.Lambda(
        lambda x: tf.math.l2_normalize(x, axis=1),
        output_shape=lambda input_shape: input_shape
    )(x)
    return Model(inputs=base_model.input, outputs=outputs)

In [8]:
base_model = create_base_model()
base_model.load_weights('/content/drive/MyDrive/ML Project/trained_base_model_4_epochs.keras')

In [9]:
print(base_model)

In [10]:
def load_and_preprocess_frame(frame_data, target_size=(224, 224)):
    nparr = np.frombuffer(frame_data, np.uint8)
    img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, target_size)
    img = img.astype(np.float32) / 255.0
    return img

def augment_frames(frames):
    # Convert to tensor if not already
    frames = tf.convert_to_tensor(frames, dtype=tf.float32)

    # Apply various augmentations
    frames = tf.image.random_brightness(frames, 0.2)
    frames = tf.image.random_contrast(frames, 0.8, 1.2)
    frames = tf.image.random_saturation(frames, 0.8, 1.2)

    # Ensure values stay in valid range
    frames = tf.clip_by_value(frames, 0.0, 1.0)
    return frames

def create_decoder(input_shape=(128,)):
    inputs = Input(shape=input_shape)
    # Add dropout and increase layer width
    x = Dense(1024, activation='relu')(inputs)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = Dense(2048, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = Dense(224 * 224 * 3, activation='sigmoid')(x)
    outputs = Reshape((224, 224, 3))(x)
    return Model(inputs=inputs, outputs=outputs)

# Create and compile decoder with explicit optimizer
decoder = create_decoder()
decoder.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
    loss='mse'
)

def generate_decoder_data(model, df, batch_size=32):
    while True:
        batch = df.sample(n=batch_size)
        frames = np.stack([load_and_preprocess_frame(frame_data) for frame_data in batch['frame_data']])
        frames = tf.convert_to_tensor(frames, dtype=tf.float32)

        # Apply augmentation to input frames
        augmented_frames = augment_frames(frames)

        # Get features from augmented frames
        features = model(augmented_frames, training=False)

        # Return features and original frames (not augmented) as targets
        yield features.numpy(), frames

# Add callbacks for monitoring and early stopping
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='loss',
        patience=5,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='loss',
        factor=0.5,
        patience=3,
        min_lr=1e-6
    ),
    tf.keras.callbacks.ModelCheckpoint(
        'best_decoder_model.keras',
        monitor='loss',
        save_best_only=True
    )
]

def train_decoder(model, decoder, df, epochs=1, steps_per_epoch=50, batch_size=32):
    generator = generate_decoder_data(model, df, batch_size)

    history = decoder.fit(
        generator,
        steps_per_epoch=steps_per_epoch,
        epochs=epochs,
        verbose=1,
        callbacks=callbacks  # Add callbacks here
    )

    return history

In [ ]:
# Train with increased epochs and batch size
history = train_decoder(
    base_model,
    decoder,
    df,
    epochs=5,
    steps_per_epoch=100,
    batch_size=32
)

Epoch 1/10
100/100 [==============================] - 609s 6s/step - loss: 0.0450 - lr: 0.0010
Epoch 2/10
100/100 [==============================] - 600s 6s/step - loss: 0.0200 - lr: 0.0010
Epoch 3/10
100/100 [==============================] - 576s 6s/step - loss: 0.0201 - lr: 0.0010
Epoch 4/10
100/100 [==============================] - 606s 6s/step - loss: 0.0198 - lr: 0.0010
Epoch 5/10
100/100 [==============================] - 572s 6s/step - loss: 0.0199 - lr: 0.0010
Epoch 6/10
100/100 [==============================] - 599s 6s/step - loss: 0.0197 - lr: 0.0010
Epoch 7/10
100/100 [==============================] - 587s 6s/step - loss: 0.0195 - lr: 0.0010
Epoch 8/10
100/100 [==============================] - 583s 6s/step - loss: 0.0191 - lr: 0.0010
Epoch 9/10
100/100 [==============================] - 572s 6s/step - loss: 0.0194 - lr: 0.0010
Epoch 10/10
 35/100 [=========>....................] - ETA: 6:24 - loss: 0.0189

In [1]:
# Save the trained decoder
decoder.save('/content/drive/MyDrive/ML Project/trained_decoder_4.keras')

NameError: name 'decoder' is not defined

In [ ]:
def preprocess_frame_for_prediction(frame_data, target_size=(224, 224)):
    img = load_and_preprocess_frame(frame_data, target_size)
    return np.expand_dims(img, axis=0)  # Add batch dimension

In [ ]:
def predict_frame(model, frame_data):
    preprocessed_frame = preprocess_frame_for_prediction(frame_data)
    prediction = model.predict(preprocessed_frame)
    return prediction

In [ ]:
import matplotlib.pyplot as plt

def predict_and_visualize_frame(model, decoder, frame_data):
    # Make prediction
    prediction = predict_frame(model, frame_data)

    # Decode the prediction
    decoded_prediction = decoder.predict(prediction)

    # Reshape and denormalize the decoded prediction
    decoded_image = (decoded_prediction[0] * 255).astype(np.uint8)

    # Display the original and predicted frames
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

    # Original frame
    original_frame = load_and_preprocess_frame(frame_data)
    ax1.imshow(original_frame)
    ax1.set_title('Original Frame')
    ax1.axis('off')

    # Predicted frame
    ax2.imshow(decoded_image)
    ax2.set_title('Predicted Frame')
    ax2.axis('off')

    plt.show()

In [ ]:
loaded_decoder = tf.keras.models.load_model('/content/drive/MyDrive/ML Project/trained_decoder_4.keras', compile=False)

# Assuming you have a frame to predict
frame_to_predict = df['frame_data'].iloc[0]  # Get the first frame from your dataframe

# Predict and visualize
predict_and_visualize_frame(base_model, loaded_decoder, frame_to_predict)